In [64]:
import sys
import subprocess
import importlib

packages = ['pandas', 'numpy','pathlib','os','glob', 'textblob']
[subprocess.check_call(['pip', 'install', pkg]) 
for pkg in packages if not importlib.util.find_spec(pkg)]

[]

In [65]:
import pandas as pd
import numpy as np

from pathlib import Path
import pathlib
import os
import glob

from textblob import TextBlob
from pathlib import Path

In [66]:
data_folder = Path(os.getcwd() + "/Desktop/IESEG_Classes/Financial_Programming/Group Project/fin_datamart-main/FP_GroupProject/data/raw/data_berka")

In [67]:
data_folder

WindowsPath('C:/Users/HP PC/Desktop/IESEG_Classes/Financial_Programming/Group Project/fin_datamart-main/FP_GroupProject/data/raw/data_berka/Desktop/IESEG_Classes/Financial_Programming/Group Project/fin_datamart-main/FP_GroupProject/data/raw/data_berka')

In [ ]:
os.chdir(data_folder)

In [69]:
# Loop through data folder and add all df's in a dict
data_dict={}
for txt_file in glob.glob("*.asc"):
    #list filenames
    filename = txt_file
    #create python tablenames
    df_name = str.replace(txt_file,".asc","_raw") 
    #read files
    df_value = pd.read_csv(filename,  delimiter=";",low_memory=False)
    #add to dict
    data_dict[df_name] = df_value

In [70]:
# Split dict into indivisual dataframes
for var in data_dict.keys():
    exec("{} = data_dict['{}']".format(var, var))

In [71]:
data_dict.keys()

dict_keys(['account_raw', 'card_raw', 'client_raw', 'disp_raw', 'district_raw', 'loan_raw', 'order_raw', 'trans_raw'])

In [72]:
# Check for null values
for keys in data_dict.keys():
    print(keys, ":", data_dict[keys].isna().sum().sum())

account_raw : 0
card_raw : 0
client_raw : 0
disp_raw : 0
district_raw : 0
loan_raw : 0
order_raw : 0
trans_raw : 2208738


In [11]:
account_raw.head()

,account_id,district_id,frequency,date
0,576,55,POPLATEK MESICNE,930101
1,3818,74,POPLATEK MESICNE,930101
2,704,55,POPLATEK MESICNE,930101
3,2378,16,POPLATEK MESICNE,930101
4,2632,24,POPLATEK MESICNE,930102


In [12]:
# Translate frequency column to english
freq_dict = {}
for text in account_raw.frequency.unique():
    freq_dict[text] =  (TextBlob(text).translate(to='en').raw)

In [13]:
freq_dict

{'POPLATEK MESICNE': 'MONTHLY FEE',
 'POPLATEK PO OBRATU': 'TURNOVER FEE',
 'POPLATEK TYDNE': 'FEE OF THE WEEK'}

In [14]:
#add translated values in dataframe
account_raw.frequency.replace({"POPLATEK MESICNE": freq_dict["POPLATEK MESICNE"],
                                "POPLATEK PO OBRATU": freq_dict["POPLATEK PO OBRATU"],
                                "POPLATEK TYDNE": freq_dict["POPLATEK TYDNE"]}, inplace=True)

In [15]:
account_raw.head()

,account_id,district_id,frequency,date
0,576,55,MONTHLY FEE,930101
1,3818,74,MONTHLY FEE,930101
2,704,55,MONTHLY FEE,930101
3,2378,16,MONTHLY FEE,930101
4,2632,24,MONTHLY FEE,930102


In [16]:
# Create column year
account_raw['year'] = account_raw['date'].astype(str).str[:2].astype(int) + 1900
df = account_raw[account_raw['year'] < 1996].copy(deep=True)
df.head()

,account_id,district_id,frequency,date,year
0,576,55,MONTHLY FEE,930101,1993
1,3818,74,MONTHLY FEE,930101,1993
2,704,55,MONTHLY FEE,930101,1993
3,2378,16,MONTHLY FEE,930101,1993
4,2632,24,MONTHLY FEE,930102,1993


In [17]:
#Add LOR
df['lor'] = 1997 - df['year']
df.head()

,account_id,district_id,frequency,date,year,lor
0,576,55,MONTHLY FEE,930101,1993,4
1,3818,74,MONTHLY FEE,930101,1993,4
2,704,55,MONTHLY FEE,930101,1993,4
3,2378,16,MONTHLY FEE,930101,1993,4
4,2632,24,MONTHLY FEE,930102,1993,4


In [18]:
# Add information about account owner
df = pd.merge(df, disp_raw[disp_raw['type'] == 'OWNER'], how='left', on='account_id')
df = pd.merge(df, client_raw, how='left', on='client_id')
df = df.rename(columns={'district_id_x':'bank_district_id',
                        'district_id_y':'client_district_id'})
df.head()

,account_id,bank_district_id,frequency,date,year,lor,disp_id,client_id,type,birth_number,client_district_id
0,576,55,MONTHLY FEE,930101,1993,4,692,692,OWNER,365111,74
1,3818,74,MONTHLY FEE,930101,1993,4,4601,4601,OWNER,350402,1
2,704,55,MONTHLY FEE,930101,1993,4,844,844,OWNER,450114,22
3,2378,16,MONTHLY FEE,930101,1993,4,2873,2873,OWNER,755324,16
4,2632,24,MONTHLY FEE,930102,1993,4,3177,3177,OWNER,380812,24


In [19]:
# Transform the birth day into year
df['birth_year'] = '19' + df['birth_number'].astype(str).str[:2]
df['birth_year'] = df['birth_year'].astype(int)

# Transform the birth day to day
df['birth_day'] = df['birth_number'].astype(str).str[-2:].astype(int)

# Extract the birth month
df['birth_month'] = df['birth_number'].astype(str).str[2:4].astype(int)

# Extract and correct the gender
df['gender'] = 'M'
df.loc[df['birth_month'] > 50, 'gender'] = 'F'

# Correct the birth month
df.loc[df['birth_month'] > 50, 'birth_month'] = df.loc[df['birth_month'] > 50, 'birth_month'] - 50
df.head()

,account_id,bank_district_id,frequency,date,year,lor,disp_id,client_id,type,birth_number,client_district_id,birth_year,birth_day,birth_month,gender
0,576,55,MONTHLY FEE,930101,1993,4,692,692,OWNER,365111,74,1936,11,1,F
1,3818,74,MONTHLY FEE,930101,1993,4,4601,4601,OWNER,350402,1,1935,2,4,M
2,704,55,MONTHLY FEE,930101,1993,4,844,844,OWNER,450114,22,1945,14,1,M
3,2378,16,MONTHLY FEE,930101,1993,4,2873,2873,OWNER,755324,16,1975,24,3,F
4,2632,24,MONTHLY FEE,930102,1993,4,3177,3177,OWNER,380812,24,1938,12,8,M


In [20]:
# Age
df['age'] = 1996 - df['birth_year']

# Age group
df['age_group'] = df['age'] // 10 * 10
df.head()

,account_id,bank_district_id,frequency,date,year,lor,disp_id,client_id,type,birth_number,client_district_id,birth_year,birth_day,birth_month,gender,age,age_group
0,576,55,MONTHLY FEE,930101,1993,4,692,692,OWNER,365111,74,1936,11,1,F,60,60
1,3818,74,MONTHLY FEE,930101,1993,4,4601,4601,OWNER,350402,1,1935,2,4,M,61,60
2,704,55,MONTHLY FEE,930101,1993,4,844,844,OWNER,450114,22,1945,14,1,M,51,50
3,2378,16,MONTHLY FEE,930101,1993,4,2873,2873,OWNER,755324,16,1975,24,3,F,21,20
4,2632,24,MONTHLY FEE,930102,1993,4,3177,3177,OWNER,380812,24,1938,12,8,M,58,50


In [21]:
loan_raw.head()

,loan_id,account_id,date,amount,duration,payments,status
0,5314,1787,930705,96396,12,8033.0,B
1,5316,1801,930711,165960,36,4610.0,A
2,6863,9188,930728,127080,60,2118.0,A
3,5325,1843,930803,105804,36,2939.0,A
4,7240,11013,930906,274740,60,4579.0,A


In [33]:
df.count()

account_id            2239
bank_district_id      2239
frequency             2239
date                  2239
year                  2239
lor                   2239
disp_id               2239
client_id             2239
type                  2239
birth_number          2239
client_district_id    2239
birth_year            2239
birth_day             2239
birth_month           2239
gender                2239
age                   2239
age_group             2239
dtype: int64

In [34]:
#add independent variables from loan df to main df
df['loan_amount'] = df.account_id.map(loan_raw.set_index('account_id')['amount'].to_dict())

In [35]:
#add independent variables from loan df to main df
df['loan_duration'] = df.account_id.map(loan_raw.set_index('account_id')['duration'].to_dict())

In [38]:
#add independent variables from loan df to main df
df['loan_status'] = df.account_id.map(loan_raw.set_index('account_id')['status'].to_dict())

In [235]:
#Filter dependent variable loan data i.e loan data for year 1997
loan_raw = loan_raw[loan_raw['date'].astype(str).str[:2].isin(['97'])]
loan_raw

,loan_id,account_id,date,amount,duration,payments,status
328,5895,4473,970103,93960,60,1566.0,C
329,7122,10365,970104,260640,36,7240.0,D
330,6173,5724,970108,232560,48,4845.0,C
331,6142,5591,970121,221880,60,3698.0,C
332,5358,2018,970121,38520,12,3210.0,A
...,...,...,...,...,...,...,...
519,6059,5196,971228,79824,12,6652.0,A
520,5799,4052,971228,106128,36,2948.0,C
521,6147,5602,971228,177504,48,3698.0,C
522,6254,6157,971228,247920,48,5165.0,C


In [236]:
loan_raw['granted'] = '1'

C:\Users\HPPC~1\AppData\Local\Temp/ipykernel_5580/3910791078.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loan_raw['granted'] = '1'


In [237]:
loan_raw.head()

,loan_id,account_id,date,amount,duration,payments,status,granted
328,5895,4473,970103,93960,60,1566.0,C,1
329,7122,10365,970104,260640,36,7240.0,D,1
330,6173,5724,970108,232560,48,4845.0,C,1
331,6142,5591,970121,221880,60,3698.0,C,1
332,5358,2018,970121,38520,12,3210.0,A,1


In [40]:
#add independent variables from transaction  df to main df
df['trans_amount'] = df.account_id.map(trans_raw.set_index('account_id')['amount'].to_dict())

In [74]:
#add independent variables from transaction  df to main df
df['trans_type'] = df.account_id.map(trans_raw.set_index('account_id')['type'].to_dict())

In [42]:
district_raw.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616,3040


In [44]:
#add independent variables from district  df to main df
df['district_name'] = df.bank_district_id.map(district_raw.set_index('A1')['A2'].to_dict())

In [46]:
df['region'] = df.bank_district_id.map(district_raw.set_index('A1')['A3'].to_dict())

In [48]:
df['avg_salary'] = df.bank_district_id.map(district_raw.set_index('A1')['A11'].to_dict())
df['unemp_95'] = df.bank_district_id.map(district_raw.set_index('A1')['A12'].to_dict())
df['unemp_96'] = df.bank_district_id.map(district_raw.set_index('A1')['A13'].to_dict())
df['no_of_enterprnr'] = df.bank_district_id.map(district_raw.set_index('A1')['A14'].to_dict())
df['crimes_95'] = df.bank_district_id.map(district_raw.set_index('A1')['A15'].to_dict())
df['crimes_96'] = df.bank_district_id.map(district_raw.set_index('A1')['A16'].to_dict())

In [75]:
df.head()

,account_id,bank_district_id,frequency,date,year,lor,disp_id,client_id,type,birth_number,...,trans_amount,district_name,region,avg_salary,unemp_95,unemp_96,no_of_enterprnr,crimes_95,crimes_96,trans_type
0,576,55,MONTHLY FEE,930101,1993,4,692,692,OWNER,365111,...,147.2,Brno - venkov,south Moravia,8743,1.88,2.43,111,3659,3894,PRIJEM
1,3818,74,MONTHLY FEE,930101,1993,4,4601,4601,OWNER,350402,...,244.7,Ostrava - mesto,north Moravia,10673,4.75,5.44,100,18782,18347,PRIJEM
2,704,55,MONTHLY FEE,930101,1993,4,844,844,OWNER,450114,...,303.6,Brno - venkov,south Moravia,8743,1.88,2.43,111,3659,3894,PRIJEM
3,2378,16,MONTHLY FEE,930101,1993,4,2873,2873,OWNER,755324,...,216.2,Jindrichuv Hradec,south Bohemia,8427,1.12,1.54,107,1874,1913,PRIJEM
4,2632,24,MONTHLY FEE,930102,1993,4,3177,3177,OWNER,380812,...,248.0,Karlovy Vary,west Bohemia,8991,1.39,2.01,128,5198,5273,PRIJEM


In [76]:
# create a dictionary to identify credit and withdrawl
type_dict = {'PRIJEM':'credit','VYDAJ':'withdrawl', 'VYBER':'withdrawl' }


In [77]:
#map dictionary to base dataframe for credit and withdrawl
df['trans_type'] = df['trans_type'].map(type_dict)


In [78]:
df.head()

,account_id,bank_district_id,frequency,date,year,lor,disp_id,client_id,type,birth_number,...,trans_amount,district_name,region,avg_salary,unemp_95,unemp_96,no_of_enterprnr,crimes_95,crimes_96,trans_type
0,576,55,MONTHLY FEE,930101,1993,4,692,692,OWNER,365111,...,147.2,Brno - venkov,south Moravia,8743,1.88,2.43,111,3659,3894,credit
1,3818,74,MONTHLY FEE,930101,1993,4,4601,4601,OWNER,350402,...,244.7,Ostrava - mesto,north Moravia,10673,4.75,5.44,100,18782,18347,credit
2,704,55,MONTHLY FEE,930101,1993,4,844,844,OWNER,450114,...,303.6,Brno - venkov,south Moravia,8743,1.88,2.43,111,3659,3894,credit
3,2378,16,MONTHLY FEE,930101,1993,4,2873,2873,OWNER,755324,...,216.2,Jindrichuv Hradec,south Bohemia,8427,1.12,1.54,107,1874,1913,credit
4,2632,24,MONTHLY FEE,930102,1993,4,3177,3177,OWNER,380812,...,248.0,Karlovy Vary,west Bohemia,8991,1.39,2.01,128,5198,5273,credit
